In [9]:
! pip install langchain
! pip install transformer
! pip install torch
! pip install gpt4all
! pip install unstructured 

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
  Obtaining dependency information for gpt4all from https://files.pythonhosted.org/packages/46/10/71bdc5cd484cdf896a7b9bf328d4818594013cfccc547ef0315d03466aba/gpt4all-1.0.8-py3-none-macosx_10_9_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 8.0 MB/s eta 0:00:008.0 MB/s eta 0:00:01
  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/e4/95/83078a61d4569afd17c86dd3741d5c176a41e79d3dd95caecfeafd3659c3/unstructured-0.8.7-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.5 MB/s eta 0:00:000:00:010:00:01
  Using cached lxml-4.9.3.tar.gz (3.6 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.2 MB/

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from string import Template
from langchain.memory import ConversationBufferMemory
from typing import List
from gpt4all import GPT4All
import argparse
import pandas as pd
from tqdm import tqdm


 33%|███▎      | 1/3 [04:35<09:10, 275.42s/it]


In [1]:
! wget "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
! unzip wiki_gameofthrones_txt1.zip

--2023-07-30 16:11:54--  https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.75.87
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.75.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1095120 (1.0M) [application/zip]
Saving to: ‘wiki_gameofthrones_txt1.zip’

wiki_gameofthrones_ 100%[===================>]   1.04M   967KB/s    in 1.1s    

2023-07-30 16:11:56 (967 KB/s) - ‘wiki_gameofthrones_txt1.zip’ saved [1095120/1095120]

Archive:  wiki_gameofthrones_txt1.zip
  inflating: 299_Rani_Mahal__TV_series_.txt  
  inflating: 133_Game_of_Thrones__Season_5__soundtrack_.txt  
  inflating: 135_Game_of_Thrones__Season_7__soundtrack_.txt  
  inflating: 26_Game_of_Thrones__Season_3__soundtrack_.txt  
  inflating: 399_For_the_Throne__Music_Inspired_by_the_HBO_Series_Game_of_Thrones.txt  
  inflating: 365_A_

In [2]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('../', glob="**/*.txt", show_progress=True)

In [3]:
docs = loader.load()

  0%|          | 0/183 [00:00<?, ?it/s]

100%|██████████| 183/183 [00:26<00:00,  6.87it/s]


In [6]:
text_content=" ".join([ i.page_content for i in docs])

In [7]:
len(text_content)

2643373

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = splitter.create_documents([i.page_content for i in docs])


In [10]:
memory = ConversationBufferMemory()
gpt4all_model = GPT4All("orca-mini-3b.ggmlv3.q4_0.bin")
chat_history = memory.load_memory_variables({})["history"]

NameError: name 'ConversationBufferMemory' is not defined

In [ ]:
def retrieve_from_database(
    input_query: str, vectorstore: Chroma, number_documents: int = 5
):
    """
    Retrieves similar documents from 'vectorstore' based on 'input_query'.
    """
    similar_docs = vectorstore.similarity_search(
        input_query, number_documents=number_documents
    )
    return similar_docs

query = str(input("Ask a question"))
context = retrieve_from_database(query, vectorstore)
prompt = get_chat_prompt(context, query, chat_history)
answer = gpt4all_model.generate(prompt=prompt)
memory.chat_memory.add_user_message(query)
memory.chat_memory.add_ai_message(answer)
